In [1]:
import sys
# print(sys.executable)
# print(sys.version)
import cv2
import cvzone

from cvzone.HandTrackingModule import HandDetector

In [2]:
def all_fingers_raised(lmList):

    if len(lmList) != 21:
        return False
        
    thumb_tip = 4
    thumb_ip = 3
    index_tip = 8
    index_ip = 7
    middle_tip = 12
    middle_ip = 11
    ring_tip = 16
    ring_ip = 15
    pinky_tip = 20
    pinky_ip = 19
    return (is_finger_raised(lmList, thumb_tip, thumb_ip) and
            is_finger_raised(lmList, index_tip, index_ip) and
            is_finger_raised(lmList, middle_tip, middle_ip) and
            is_finger_raised(lmList, ring_tip, ring_ip) and
            is_finger_raised(lmList, pinky_tip, pinky_ip))

def is_finger_raised(lmList, tip, phalanx):
    return lmList[tip][1] < lmList[phalanx][1]

In [18]:
def set_up(width, height):
    cap = cv2.VideoCapture(0)
    cap.set(3, width)
    cap.set(4, height)
    return cap


def is_box_dragged(distance, lmList, center_x, center_y, width, height):
    if distance < 40:
        cursor = lmList[8][:2]
        if center_x - width //2 < cursor[0] < center_x+width //2 and center_y - height //2 < cursor[1] < center_y+height //2:
            rectangle_color = (255, 255, 255)
            center_x, center_y = cursor


def detect():
    
    hand_detector = HandDetector(detectionCon=0, maxHands=1)
    width, height = 700, 600
    cap = set_up(width, height)
    rectangle_color = (255, 0, 255)
    
    center_x, center_y, width, height = 100, 100, 100, 100
    
    
    while True:
        
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        hands, _ = hand_detector.findHands(frame)
    
        if hands:
            # x and y of tip of your finger
    
            lmList = hands[0]['lmList']
    
            index_finger = lmList[8][:2]
            middle_finger = lmList[12][:2]
            distance = ((index_finger[0] - middle_finger[0])**2 + (index_finger[1] - middle_finger[1])**2)**0.5
            # index_middle_distance, _, _ = hand_detector.findDistance(8, 12, frame)
    
            # rectangle turns red when all fingers are raised
            if all_fingers_raised(lmList):
                cv2.rectangle(frame, (center_x - width//2, center_y - height//2),
                      (center_x+width//2,center_y+height//2), rectangle_color, cv2.FILLED)
            
            is_box_dragged(distance, lmList, center_x, center_y, width, height)
    
        
        cv2.imshow("cam", frame)
        
        wait_key = cv2.waitKey(1)
        if wait_key == ord("q"):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [19]:
detect()

I0000 00:00:1726244787.939463   26615 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726244787.941536   27569 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-1pop1~1711635559~22.04~7a9f319), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
W0000 00:00:1726244787.987765   27562 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726244788.020976   27560 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
